# 6장 6-9 연습문제


|학과|학번|이름|작성일|
|----|----|----|----|
|컴퓨터공학과|202111615|함기은|2025-09-23|

* 파이썬 언어에서 다음에 대해 조사하고 예제코드를 실행하여 보라
• 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들
• 2. 파이썬에서 멀티쓰레드 구현하는 기법
• 3. 멀티쓰레드에서 공유 변수에 대한 Critical Section 을 보호하는 기법

1. 파이썬으로 싱글톤(Singleton) 패턴을 구현하는 방법들
싱글톤 패턴은 특정 클래스의 인스턴스가 프로그램 내에서 오직 하나만 생성되도록 보장하는 디자인 패턴입니다. 주로 데이터베이스 연결, 로거, 설정 관리 등 공유 자원에 접근할 때 사용됩니다.

방법 1: __new__ 메소드 사용
가장 일반적인 방법으로, 클래스의 __new__ 메소드를 오버라이드하여 인스턴스 생성을 제어합니다.

```Python
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance

# 테스트
s1 = Singleton()
s2 = Singleton()

print(f"s1과 s2는 같은 객체인가? {s1 is s2}") # 출력: True
```

방법 2: 데코레이터(Decorator) 사용
데코레이터를 사용하여 싱글톤 로직을 클래스 정의와 분리할 수 있어 더 깔끔합니다.

```Python
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("데이터베이스 객체 생성")

# 테스트
db1 = Database()
db2 = Database()

print(f"db1과 db2는 같은 객체인가? {db1 is db2}") # 출력: True
# "데이터베이스 객체 생성"은 한 번만 출력됩니다.
```

방법 3: 메타클래스(Metaclass) 사용
메타클래스를 사용하면 클래스 생성 자체를 제어할 수 있어 더욱 강력한 싱글톤 구현이 가능합니다.

```Python
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    pass

# 테스트
logger1 = Logger()
logger2 = Logger()

print(f"logger1과 logger2는 같은 객체인가? {logger1 is logger2}") # 출력: True
```

2. 파이썬에서 멀티쓰레드(Multi-thread) 구현하는 기법
멀티쓰레딩은 하나의 프로세스 내에서 여러 개의 실행 흐름(쓰레드)을 만들어 작업을 동시에 처리하는 기법입니다. I/O 작업(파일 읽기/쓰기, 네트워크 통신 등)이 많은 프로그램의 응답성을 높이는 데 효과적입니다. 파이썬에서는 threading 모듈을 사용합니다.

방법 1: threading.Thread 클래스에 함수 전달
가장 간단한 방법으로, 쓰레드에서 실행할 함수를 Thread 객체의 target 인자로 전달합니다.

```Python
import threading
import time

def worker(name, duration):
    print(f"쓰레드 {name}: 시작")
    time.sleep(duration)
    print(f"쓰레드 {name}: 종료")

# 쓰레드 생성
t1 = threading.Thread(target=worker, args=("A", 2))
t2 = threading.Thread(target=worker, args=("B", 1))

# 쓰레드 시작
t1.start()
t2.start()

# 메인 쓰레드가 자식 쓰레드들이 끝날 때까지 기다림
t1.join()
t2.join()

print("모든 쓰레드 작업 완료")
```

방법 2: threading.Thread 클래스 상속
더 복잡한 로직이나 상태 관리가 필요할 때, threading.Thread를 상속받아 run 메소드를 오버라이드합니다.

```Python
import threading
import time

class MyThread(threading.Thread):
    def __init__(self, name, duration):
        super().__init__()
        self.name = name
        self.duration = duration

    def run(self):
        print(f"쓰레드 {self.name}: 시작")
        time.sleep(self.duration)
        print(f"쓰레드 {self.name}: 종료")

# 쓰레드 생성 및 시작
t1 = MyThread("C", 1.5)
t2 = MyThread("D", 2.5)
t1.start()
t2.start()

t1.join()
t2.join()

print("모든 쓰레드 작업 완료")
```

3. 멀티쓰레드에서 공유 변수에 대한 Critical Section 보호 기법
여러 쓰레드가 공유 변수(자원)에 동시에 접근하여 수정할 때, 경쟁 상태(Race Condition)가 발생하여 데이터 무결성이 깨질 수 있습니다. **Critical Section(임계 구역)**은 공유 자원에 접근하는 코드 영역을 의미하며, 이 영역은 한 번에 하나의 쓰레드만 접근하도록 보호해야 합니다.

Lock (또는 RLock) 사용
Lock은 가장 기본적인 동기화 기법입니다. acquire() 메소드로 락을 획득하고, release() 메소드로 락을 해제합니다. 락이 해제되기 전까지 다른 쓰레드는 acquire()에서 대기합니다.

```Python
import threading

shared_number = 0
lock = threading.Lock()

def increment():
    global shared_number
    for _ in range(1000000):
        lock.acquire() # 락 획득 (임계 구역 시작)
        shared_number += 1
        lock.release() # 락 해제 (임계 구역 끝)

# with 문을 사용하면 더 안전하게 락을 관리할 수 있습니다.
def increment_with_statement():
    global shared_number
    for _ in range(1000000):
        with lock: # with 블록에 들어가면 acquire(), 나오면 release() 자동 호출
            shared_number += 1


threads = []
for i in range(5):
    # target을 increment_with_statement로 변경하여 테스트 가능
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

# 락을 사용하지 않으면 5000000보다 작은 값이 나올 확률이 높습니다.
# 락을 사용하면 정확히 5000000이 출력됩니다.
print(f"최종 결과: {shared_number}")
```

RLock (재진입 가능 락)은 이미 락을 획득한 쓰레드가 다시 해당 락을 획득하려 할 때 교착 상태(Deadlock)에 빠지지 않도록 합니다. 재귀적인 함수 호출 등에서 유용합니다.